In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
from onehotencoder import OneHotEncoder_int
import os
import pandas as pd
from root_pandas import *

Welcome to JupyROOT 6.24/06


In [4]:

hzg = uproot.open('./datasets/ZGToLLG.root')
hzgtree = hzg['passedEvents']
arrays = hzgtree.arrays(["H_mass"], library="pd")
#new_tree = uproot.newtree({"existing_branch": hzgtree["H_mass"].interpretation, "new_branch": "float32"})
inputtmp = pd.DataFrame(arrays)
#hzgtree.extend({"H_mass_transfered":np.array([1,2,3,4,5])})
#inputtmp = pd.DataFrame(arrays)
#inputtmp.to_root('test.root', key='inclusive', mode='a', index=False)
inputtmp["H_mass_transfer"] = np.array([1]*1043837)

#inputtmp
inputtmp.to_root('test.root', key='inclusive', mode='a', index=False)

In [5]:
inputtmp

,H_mass,H_mass_transfer
0,146.910324,1
1,154.971008,1
2,119.438431,1
3,120.820694,1
4,168.040176,1
...,...,...
1043832,141.645844,1
1043833,133.707458,1
1043834,169.091354,1
1043835,143.190140,1


In [3]:
featurevars = ['H_mass', 'H_pt', 'Z_pt', 'gamma_pt', 'gamma_mvaID', 'n_jets', 'n_leptons', 'regions']
rootfile_target='./datasets/data.root'
rootfile_source='./datasets/ZGToLLG.root'

def prepdata(filename):
    hzg = uproot.open(filename)
    hzgtree = hzg['passedEvents']
    iscategorical = [False, False, False, False, False, False, False, True]
    _onehotencoder = OneHotEncoder_int(iscategorical)

    arrays = hzgtree.arrays(featurevars, library="pd")
    inputtmp = pd.DataFrame(arrays)

    inputnumpy = inputtmp.to_numpy(dtype=np.float32)
    inputs = _onehotencoder.encode(inputnumpy)
    ncats = _onehotencoder.ncats
    ncat_per_feature = _onehotencoder.categories_per_feature

    meanslist = []
    sigmalist = []
    currentcolumn = 0
    for ifeat, ncatfeat in zip(range(inputtmp.shape[1]), ncat_per_feature):
        if ncatfeat == 0: # fir float features, get mean and sigma
            mean = np.mean(inputnumpy[:, currentcolumn], axis=0, dtype=np.float32).reshape(1,1)
            meanslist.append(mean)
            sigma = np.std(inputnumpy[:, currentcolumn], axis=0, dtype=np.float32).reshape(1,1)
            sigmalist.append(sigma)
            currentcolumn += 1
        else: # categorical features do not get changed
            mean = np.zeros(shape=(1, ncatfeat), dtype=np.float32) 
            meanslist.append(mean)
            sigma = np.ones(shape=(1, ncatfeat), dtype=np.float32)
            sigmalist.append(sigma)
            currentcolumn += ncatfeat

    inputmeans = np.hstack(meanslist)
    inputsigma = np.hstack(sigmalist)

    normedinputs = (inputs-inputmeans) / inputsigma

    return inputtmp, normedinputs, inputmeans, inputsigma, ncat_per_feature

In [6]:
rawinputs_target, normedinputs_target, inputmeans_target, inputsigma_target, ncat_per_feature_target = prepdata(rootfile_target)
rawinputs_source, normedinputs_source, inputmeans_source, inputsigma_source, ncat_per_feature_source = prepdata(rootfile_source)
print(ncat_per_feature_target)
inputdim = 7
ncat_per_feature_target = ncat_per_feature_target[0:inputdim]
conddim = normedinputs_target.shape[1] - inputdim
print(conddim)

[0, 0, 0, 0, 0, 0, 0, 4]
4


In [7]:
issignal_target = (rawinputs_target['regions']==0) # signal_selection 
issignal_source = (rawinputs_source['regions']==0) # signal_selection 
isbackground_target = ~issignal_target
isbackground_source = ~issignal_source
bkgnormed_target = normedinputs_target[isbackground_target]
bkgnormed_source = normedinputs_source[isbackground_source]
bkg_target = rawinputs_target[isbackground_target]
bkg_source = rawinputs_source[isbackground_source]   
#xmax = np.reshape(inputmeans + 5* inputsigma, inputmeans.shape[1])


In [8]:
ncol=4 # for plots below
condlist = {
            "SR": [[1., 0., 0., 0., ]],
            "CR1": [[0., 1., 0., 0., ]],
            "CR2": [[0., 0., 1., 0., ]],
            "CR3": [[0., 0., 0., 1., ]]
            }

select_data_target = {
    "SR": (rawinputs_target['regions']==0),
    "CR1": (rawinputs_target['regions']==1),
    "CR2": (rawinputs_target['regions']==2),
    "CR3": (rawinputs_target['regions']==3)
}

select_data_source = {
    "SR": (rawinputs_source['regions']==0),
    "CR1": (rawinputs_source['regions']==1),
    "CR2": (rawinputs_source['regions']==2),
    "CR3": (rawinputs_source['regions']==3)
}

select_data_source_transfered = {}

plottextlist=['SR','CR1','CR2','CR3']



In [9]:
from ABCD_dnn_mmd import ABCDdnn

LRrange3 = [0.00001, 0.00001, 20000, 0]

nafdim=128
depth=2
batchsize=512
seed=101

m = ABCDdnn(ncat_per_feature_target, inputdim, minibatch=batchsize, conddim=conddim, LRrange=LRrange3, \
        beta1=0.9, beta2=0.999, nafdim=nafdim, depth=depth, savedir='test', permute=True, retrain=False, seed=seed)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 11)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 7)            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.convert_to_tensor (TFOpLambd (None, 7)            0           tf.__operators__.getitem_1[0][0] 
__________________________________________________________________________________________________
tf.compat.v1.gather (TFOpLambda (None, 7)            0           tf.convert_to_tensor[0][0]       
______________________________________________________________________________________________

AssertionError: No checkpoint specified (save_path=None); nothing is being restored.

2024-02-21 17:09:57.372095: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-21 17:09:57.386409: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lxslc713.ihep.ac.cn): /proc/driver/nvidia/version does not exist
2024-02-21 17:09:57.569022: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
import tensorflow as tf
#model_file = "/hpcfs/cepc/higgsgpu/wangzebing/ABCDnn/ABCDnn/hzg_nafnodropout256_batchsize4096_depth8_seed123_9_3_LR3mmd1e-05to1e-05_nP7/hyperparams.pkl"
model_file = "/hpcfs/cepc/higgsgpu/wangzebing/ABCDnn/ABCDnn/abcdnn_HZG"

model = m.model
checkpoint = tf.train.Checkpoint(model=model) 
checkpoint.restore(tf.train.latest_checkpoint(model_file))

In [14]:
#xin = normedinputs_source[select_data_source["CR1"]][:1,:]
#model.predict(xin)
normedinputs_source[select_data_source["CR1"]][:1,:]
inputsigma_target

array([[19.749079  , 31.70903   , 32.367634  , 12.673199  ,  0.46424174,
         0.8798654 ,  0.06757355,  1.        ,  1.        ,  1.        ,
         1.        ]], dtype=float32)

In [20]:
a = np.array([[1,2,3]], dtype=np.float32)
b = np.array([[0.5,0.5,0.5]], dtype=np.float32)
a+b

array([[1.5, 2.5, 3.5]], dtype=float32)

In [7]:

#xin = normedinputs_source[select_data_source["CR2"]][0:1]
#print(cond_to_append, xin, condlist["CR2"])
#print(xin)
#m.model.predict(xin)
batchsize = 4096
minibatch = batchsize

#for r in plottextlist:
for r in ["CR1"]:
        select_data_source_transfered[r] = []

        transferedlist = []

        #xin = normedinputs_source[select_data_source[r]]
        xin = normedinputs_source[select_data_source[r]][:10,:]
        xgen = model.predict(xin)
        transferedlist.append(xgen)

        transfered_data_norm = np.vstack(transferedlist)
        transfered_data = transfered_data_norm * inputsigma_source[:, :inputdim] + inputmeans_source[:, :inputdim]
        ntransfered_data = transfered_data.shape[0]

        select_data_source_transfered[r].append(transfered_data)




AttributeError: 'list' object has no attribute 'predict'